In [1]:
import sqlite3
import os
import mlflow

# Specify the path where you want to store the database
db_path = '../mlflow-db/mlflow.db'

# Check if the database already exists
if os.path.exists(db_path):
    print(f"Database already exists at {db_path}")
else:
    # Connect to the SQLite database (it will create the file if it doesn't exist)
    conn = sqlite3.connect(db_path)
    print(f"Database created at {db_path}")
    # Close the connection
    conn.close()


Database created at ../mlflow-db/mlflow.db


## Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use [conda environments](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-envs), and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

What's the version that you have?

**<span style="color:green">mlflow, version 2.22.0</span>**


In [14]:
!mlflow --version

mlflow, version 2.22.0


## Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset to predict the duration of each trip. 

Download the data for January, February and March 2023 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](homework) to preprocess the data.

The script will:

* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2023 data),
* save the preprocessed datasets and the `DictVectorizer` to disk.

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to `OUTPUT_FOLDER`?

* 1
* 3 
* **<span style="color:green">4</span>**
* 7


In [15]:
!python preprocess_data.py --raw_data_path ../data/ --dest_path "../Homework-02-Prepocess-Output"

In [16]:
!dir /a-d /b "..\Homework-02-Prepocess-Output" | find /c /v ""

4


## Q3. Train a model with autolog

We will train a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder [`homework`](homework). 

The script will:

* load the datasets produced by the previous step,
* train the model on the training set,
* calculate the RMSE score on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

What is the value of the `min_samples_split` parameter:

* **<span style="color:green">2</span>**
* 4
* 8
* 10


In [18]:
!python train.py

Experiment '02-Experiment-Tracking' already exists.
Tracking URI:- sqlite:///e:\GitHub\DTC MLOps\mlflow-db\mlflow.db
Artifact Path:- file:///E:/GitHub/DTC MLOps/mlflow-artifacts/02-Experiment-Tracking
Artifact Location for '02-Experiment-Tracking': file:///E:/GitHub/DTC MLOps/mlflow-artifacts/02-Experiment-Tracking


2025/05/14 11:08:33 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/05/14 11:08:36 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '4b1d5efe256b493092324d7b0a38ff86', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


In [19]:
experiment_name = "02-Experiment-Tracking"
run_id = "2e7f53e68e844134982a419e6900f97c"
# Get the experiment ID from the experiment name
mlflow.set_tracking_uri("sqlite:///E:/GitHub/DTC MLOps/mlflow-db/mlflow.db")
experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment:
    experiment_id = experiment.experiment_id

    # Get the run details using the run ID
    run = mlflow.get_run(run_id)

    # Access parameters from the run
    parameters = run.data.params
    print(f"Parameters for run {run_id}:")
    param = 'min_samples_split'
    value=parameters['min_samples_split']
    print(f"{param}: {value}")
else:
    print(f"Experiment '{experiment_name}' not found.")

Parameters for run 2e7f53e68e844134982a419e6900f97c:
min_samples_split: 2


## Q4. Launch the tracking server locally

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

Your task is to:

* launch the tracking server on your local machine,
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

* **<span style="color:green">default-artifact-root</span>**
* `serve-artifacts`
* `artifacts-only`
* `artifacts-destination`


## Q5. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `hyperopt`. 
We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization,
* the RMSE obtained on the validation set (February 2023 data).

What's the best validation RMSE that you got?

* 4.817
* **<span style="color:green">5.335</span>**
* 5.818
* 6.336


In [3]:
!python hpo.py

Experiment 'random-forest-hyperopt' already exists.
Artifact Location for 'random-forest-hyperopt': file:///E:/GitHub/DTC MLOps/mlflow-artifacts/random-forest-hyperopt

100%|██████████| 15/15 [01:58<00:00,  7.93s/trial, best loss: 5.335419588556921]


## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. 
In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. 
After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right `model_uri` in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

* 5.060
* **<span style="color:green">5.567</span>**
* 6.061
* 6.568

In [4]:
!python register_model.py

Experiment 'random-forest-best-models' created with ID: 2
Best run ID: 914c2018ffdb4ae7983fa0d986f2e95f
Lowest Test RMSE: 5.567408012462019
Name: RandomForestRegressionModel
Version: 1


Successfully registered model 'RandomForestRegressionModel'.
Created version '1' of model 'RandomForestRegressionModel'.
